# Description

We have datasets corresponding to a **list of health inspections in establishments** (restaurants, supermarkets, etc.), along with their respective health risk. We have another dataset that shows a **description of said risk**.

**The goal is to load these datasets under specific requirements and manipulate them according to the instructions of each exercise.**

All necessary operations are described in the exercises, although additional tasks carried out by the student on their own initiative will be appreciated. The use of the DataFrame API will also be valued.

# Download Datasets

In [0]:
%sh 
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/food_inspections_lite.csv'
curl -O 'https://raw.githubusercontent.com/masfworld/datahack_docker/master/zeppelin/data/risk_description.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 49 7361k   49 3609k    0     0  6901k      0  0:00:01 --:--:--  0:00:01 6888k
100 7361k  100 7361k    0     0  12.8M      0 --:--:-- --:--:-- --:--:-- 12.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   176  100   176    0     0   1035      0 --:--:-- --:--:-- --:--:--  1041


In [0]:
# Copy the local file food_inspections_lite.csv from the driver node to DBFS (Databricks File System) under /dataset/
dbutils.fs.cp('file:/databricks/driver/food_inspections_lite.csv','dbfs:/dataset/food_inspections_lite.csv')

# Copy the local file risk_description.csv from the driver node to DBFS (Databricks File System) under /dataset/
dbutils.fs.cp('file:/databricks/driver/risk_description.csv','dbfs:/dataset/risk_description.csv')

Out[2]: True

In [0]:
# List all files and directories under the DBFS directory /dataset/
dbutils.fs.ls('/dataset/')

Out[3]: [FileInfo(path='dbfs:/dataset/Coral_cover_data.csv', name='Coral_cover_data.csv', size=5161518, modificationTime=1720865295000),
 FileInfo(path='dbfs:/dataset/OnlineRetail.csv', name='OnlineRetail.csv', size=45038728, modificationTime=1720865301000),
 FileInfo(path='dbfs:/dataset/bank.csv', name='bank.csv', size=461474, modificationTime=1720552654000),
 FileInfo(path='dbfs:/dataset/books/', name='books/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/dataset/characters.csv', name='characters.csv', size=5462, modificationTime=1720592921000),
 FileInfo(path='dbfs:/dataset/departuredelays.csv', name='departuredelays.csv', size=33396236, modificationTime=1720865297000),
 FileInfo(path='dbfs:/dataset/food_inspections_lite.csv', name='food_inspections_lite.csv', size=7538077, modificationTime=1720877383000),
 FileInfo(path='dbfs:/dataset/frankenstein.txt', name='frankenstein.txt', size=421623, modificationTime=1720592921000),
 FileInfo(path='dbfs:/dataset/loan.csv', name='loan.c

We do a `head` to see the content of the datasets:

In [0]:
dbutils.fs.head("dbfs:/dataset/food_inspections_lite.csv")

[Truncated to first 65536 bytes]
Out[4]: 'Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location\n2373596,COUNTY BBQ,COUNTY BBQ,2732781,Restaurant,Risk 3 (Low),1352 W TAYLOR ST ,CHICAGO,IL,60607,06/11/2020,License,Fail,,41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"\n2373587,KIDZ CREATIVE CORNER,KIDZ CREATIVE CORNER,2555611,Daycare Above and Under 2 Years,Risk 1 (High),4259 N WESTERN AVE ,CHICAGO,IL,60618,06/11/2020,License Re-Inspection,Pass,,41.95944802136064,-87.68848155320953,"(-87.68848155320953, 41.95944802136064)"\n2373578,DSD DELI,DSD DELI,2626186,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE ,CHICAGO,IL,60630,06/11/2020,License Re-Inspection,Pass,,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"\n2373575,DSD DELI,DSD DELI,2626178,DELI/GROCERY,Risk 1 (High),5205 N MILWAUKEE ,CHICAGO,IL,60630,06/11/2020,License Re-

In [0]:
dbutils.fs.head("dbfs:/dataset/risk_description.csv")

Out[5]: 'risk_id,description\n1,Este riesgo significar la clausura inmediata del local\n2,Este riesgo está cerca de convertirse en la clausura del local\n3,Necesita una mejora importante\n'

# Exercise 1
---

1. **Create two dataframes, one from the file `food_inspections_lite.csv` and another from `risk_description.csv`.**
2. **Convert these two dataframes to delta tables.**


Create the dataframe from `food_inspections_lite.csv`:

In [0]:
# Cargar el archivo food_inspections_lite.csv en un DataFrame
file_path_food = '/dataset/food_inspections_lite.csv'

food_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(file_path_food)

Rename the columns and verify that the schema is adequate:

In [0]:
# Rename the "License #" column to "License number".
food_df = food_df.withColumnRenamed("License #", "License number")

# Rename the columns by replacing spaces with underscores.
new_columns = [col_name.replace(" ", "_") for col_name in food_df.columns]

# Rename the columns in the DataFrame
food_df = food_df.toDF(*new_columns)

# Verify new column names
food_df.printSchema()

root
 |-- Inspection_ID: integer (nullable = true)
 |-- DBA_Name: string (nullable = true)
 |-- AKA_Name: string (nullable = true)
 |-- License_number: integer (nullable = true)
 |-- Facility_Type: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: integer (nullable = true)
 |-- Inspection_Date: date (nullable = true)
 |-- Inspection_Type: string (nullable = true)
 |-- Results: string (nullable = true)
 |-- Violations: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [0]:
# Display the first 3 lines of the DataFrame food_df
food_df.show(3, truncate=False)

+-------------+--------------------+--------------------+--------------+-------------------------------+-------------+-------------------+-------+-----+-----+---------------+---------------------+-------+----------+-----------------+------------------+---------------------------------------+
|Inspection_ID|DBA_Name            |AKA_Name            |License_number|Facility_Type                  |Risk         |Address            |City   |State|Zip  |Inspection_Date|Inspection_Type      |Results|Violations|Latitude         |Longitude         |Location                               |
+-------------+--------------------+--------------------+--------------+-------------------------------+-------------+-------------------+-------+-----+-----+---------------+---------------------+-------+----------+-----------------+------------------+---------------------------------------+
|2373596      |COUNTY BBQ          |COUNTY BBQ          |2732781       |Restaurant                     |Risk 3 (Low) |135

We are going to clean up the `Risk` column that will be used later in the exercises. To do this, let's list the values it has and the count of each one:

In [0]:
# Grouping by 'Risk' column and counting the occurrences
risk_counts = food_df.groupBy('Risk').count()

# Showing the results
risk_counts.show()

+---------------+-----+
|           Risk|count|
+---------------+-----+
|           null|    5|
|  Risk 1 (High)| 7215|
|            All|    9|
|Risk 2 (Medium)| 1812|
|   Risk 3 (Low)|  959|
+---------------+-----+



We see that there are `null` values and `all` values. The correct values are `Risk 1 (High)`, `Risk 2 (Medium)`, and `Risk 3 (Low)`. Therefore, we will make the following modifications:
- We will remove the `null` values since we are unable to trace them to any of the correct values.
- We will change the `All` values to `Risk 1 (High)`, considering that "All" means they have received the highest risk score.

In [0]:
# Drop rows where the 'Risk' column has null values
df_food_drop_risk_nulls = food_df.dropna(subset=['Risk'])

In [0]:
from pyspark.sql.functions import col, when
# Replacing "All" with "Risk 1 (High)"
food_df_clean = df_food_drop_risk_nulls.withColumn('Risk', when(col('Risk') == 'All', 'Risk 1 (High)').otherwise(col('Risk')))

In [0]:
# Group by 'Risk' and count the occurrences
risk_counts = food_df_clean.groupBy('Risk').count()

# Show the results
risk_counts.show()

+---------------+-----+
|           Risk|count|
+---------------+-----+
|  Risk 1 (High)| 7224|
|Risk 2 (Medium)| 1812|
|   Risk 3 (Low)|  959|
+---------------+-----+



Create the dataframe from `risk_description.csv`:

In [0]:
# Load the risk_description.csv file in another DataFrame
file_path_risk = '/dataset/risk_description.csv'
risk_df = spark.read.format("csv") \
  .option("sep", ",") \
  .option("inferSchema", "true") \
  .option("header", "true") \
  .load(file_path_risk)

In [0]:
# Show all lines of the DataFrame risk_df
risk_df.display()

risk_id,description
1,Este riesgo significar la clausura inmediata del local
2,Este riesgo está cerca de convertirse en la clausura del local
3,Necesita una mejora importante


Conversion of the DataFrame `food_df_clean` to a Delta Table:

In [0]:
from delta.tables import *
from pyspark.sql.functions import *

# Define the Delta Lake path for the 'food' table and remove the directory at FOOD_DELTA_PATH recursively, if it exists
FOOD_DELTA_PATH = "/mnt/delta/food"
dbutils.fs.rm(FOOD_DELTA_PATH, recurse=True)

# Write the DataFrame to the specified Delta path
food_df_clean.write.format("delta").save(FOOD_DELTA_PATH)

# Drop the table if it already exists
spark.sql("DROP TABLE IF EXISTS food")

# Create a Delta table named 'food' using the data saved at FOOD_DELTA_PATH
spark.sql("CREATE TABLE food USING DELTA LOCATION \'" + FOOD_DELTA_PATH + "\'")

Out[15]: DataFrame[]

In [0]:
%sql
SELECT *
FROM 
  food
LIMIT 3

Inspection_ID,DBA_Name,AKA_Name,License_number,Facility_Type,Risk,Address,City,State,Zip,Inspection_Date,Inspection_Type,Results,Violations,Latitude,Longitude,Location
2373596,COUNTY BBQ,COUNTY BBQ,2732781,Restaurant,Risk 3 (Low),1352 W TAYLOR ST,CHICAGO,IL,60607,2020-06-11,License,Fail,null,41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"
2373587,KIDZ CREATIVE CORNER,KIDZ CREATIVE CORNER,2555611,Daycare Above and Under 2 Years,Risk 1 (High),4259 N WESTERN AVE,CHICAGO,IL,60618,2020-06-11,License Re-Inspection,Pass,null,41.95944802136064,-87.68848155320953,"(-87.68848155320953, 41.95944802136064)"
2373578,DSD DELI,DSD DELI,2626186,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE,CHICAGO,IL,60630,2020-06-11,License Re-Inspection,Pass,null,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"


In [0]:
%sql
SELECT COUNT(*) AS total_rows FROM food

total_rows
9995


Conversion of the DataFrame `risk_df` to a Delta Table:

In [0]:
# Define the Delta Lake path for the 'risk' table and remove the directory at RISK_DELTA_PATH recursively, if it exists
RISK_DELTA_PATH = "/mnt/delta/risk"
dbutils.fs.rm(RISK_DELTA_PATH, recurse=True)

# Write the DataFrame to the specified Delta path
risk_df.write.format("delta").save(RISK_DELTA_PATH)

# Drop the table if it already exists
spark.sql("DROP TABLE IF EXISTS risk")

# Create a Delta table named 'risk' using the data saved at RISK_DELTA_PATH
spark.sql("CREATE TABLE risk USING DELTA LOCATION \'" + RISK_DELTA_PATH + "\'")

Out[18]: DataFrame[]

In [0]:
%sql
SELECT *
FROM 
  risk

risk_id,description
1,Este riesgo significar la clausura inmediata del local
2,Este riesgo está cerca de convertirse en la clausura del local
3,Necesita una mejora importante


# Exercise 2
**Obtain the number of distinct inspections with high `Risk 1 (High)`.**

---



In [0]:
%sql
SELECT COUNT(DISTINCT Inspection_ID) AS num_inspections_high_risk
FROM food
WHERE Risk = 'Risk 1 (High)'

num_inspections_high_risk
7224



# Exercise 3
**From the dataframes loaded above, obtain a table with the following columns:<br>**
1. `DBA Name`
2. `Facility Type`
3. `Risk`
4. `Risk description`

---
I will use PySpark's `join` function to combine both DataFrames based on the `Risk` column.

In [0]:
# Create a new column in food_df_clean to map the values of Risk to risk_id
food_df_mapped = food_df_clean.withColumn("risk_id", 
                                    when(food_df_clean["Risk"].contains("High"), 1)
                                    .when(food_df_clean["Risk"].contains("Medium"), 2)
                                    .when(food_df_clean["Risk"].contains("Low"), 3)
                                    .otherwise(None))

# Perform the link between food_df_mapped and risk_df
combined_df = food_df_mapped.join(risk_df, food_df_mapped["risk_id"] == risk_df["risk_id"], "inner") \
                            .select(food_df_mapped["DBA_Name"],
                                    food_df_mapped["Facility_Type"],
                                    food_df_mapped["Risk"],
                                    risk_df["description"].alias("Risk_description"))

# Show the result
combined_df.show(10, truncate=False)

+--------------------+-------------------------------+---------------+--------------------------------------------------------------+
|DBA_Name            |Facility_Type                  |Risk           |Risk_description                                              |
+--------------------+-------------------------------+---------------+--------------------------------------------------------------+
|COUNTY BBQ          |Restaurant                     |Risk 3 (Low)   |Necesita una mejora importante                                |
|KIDZ CREATIVE CORNER|Daycare Above and Under 2 Years|Risk 1 (High)  |Este riesgo significar la clausura inmediata del local        |
|DSD DELI            |DELI/GROCERY                   |Risk 3 (Low)   |Necesita una mejora importante                                |
|DSD DELI            |DELI/GROCERY                   |Risk 1 (High)  |Este riesgo significar la clausura inmediata del local        |
|COUNTY BBQ          |Restaurant                     |Risk 1 (

# Exercise 4
**Access the Spark UI to view the execution plan of the previous exercise (exercise 3). Describe each of the pieces/boxes that make up the execution plan (a brief one-line description per box will be sufficient).**

---
In Apache Spark, a "**job**" refers to a unit of work that is sent to the cluster to be executed. Each job can consist of one or more "**stages**", where a stage is a processing phase that includes a set of tasks that can be executed in parallel on the cluster nodes.

The execution of exercise 3 indicates that Spark has executed two **jobs**:
- **Job 41 (Stages: 1/1)**: Corresponds to the creation and transformation of the `food_df_mapped` DataFrame using `withColumn` and the `when` conditions.

- **Job 42 (Stages: 1/1, 1 skipped)**: Corresponds to the join operation (`join`) between `food_df_mapped` and `risk_df`, followed by the column selection. The "1 skipped" indicates that Spark has optimized the execution by detecting that some data was already available in cache or in the same state needed for the previous job.


## Job 41 - withColumn

![Details for Job 41](https://github.com/Ubikitina/Spark-Essentials/blob/main/Notebooks/img/04_01.png?raw=true)

![Details for Stage 50](https://github.com/Ubikitina/Spark-Essentials/blob/main/Notebooks/img/04_02.png?raw=true)


- **Scan csv (CSV file scan):** Spark starts by scanning the input CSV file (`food_inspections_lite.csv`) to read the data into an RDD. 
  - `FileScanRDD` represents the RDD created from reading the CSV file.
  - `MapPartitionsRDD`: represents the RDD resulting from partitioning the data read from the CSV file.

- **WholeStageCodegen (Single-stage code generation optimization):** is a physical query optimization in Spark SQL that merges multiple physical operators into a single Java function. Simply put, in this step, the calculations written in DataFrames are computed to generate the Java code to build the underlying RDDs, optimizing the execution of the transformations defined in the code (`withColumn`).

- **Exchange (Data exchange or redistribution):** data partitions are exchanged or redistributed to ensure that the `withColumn` transformation is correctly applied across all data partitions.




## Job 42 - Join

![Details for Job 42](https://github.com/Ubikitina/Spark-Essentials/blob/main/Notebooks/img/04_03.png?raw=true)

![Details for Stage 52](https://github.com/Ubikitina/Spark-Essentials/blob/main/Notebooks/img/04_04.png?raw=true)



**Stage 51 (skipped):** This stage was skipped in the DAG log, possibly because it was already executed previously. It is a DAG very similar to the one explained earlier.

**Stage 52:**
- **Scan csv:** Reading the `risk_df` file.
  - `FileScanRDD`: Represents the RDD of the CSV file read and processed as an RDD.
  - `MapPartitionsRDD`: Represents the RDD resulting from partitioning the data read from the CSV file.
- **ShuffleQueryStage:** Receives the data from Stage 51 and performs a shuffle operation to organize and prepare them for the subsequent Join operation.
- **WholeStageCodegen:** This is the code optimization stage, improving the efficiency of executing the `join`. It includes several `MapPartitionsRDD` stages to organize the data into partitions and a `CartesianRDD` stage corresponding to the Join itself, as the Cartesian transformation generates a Cartesian product of two RDDs.


# Exercise 5
**1. For each establishment (column `DBA Name`) and its result (column `Results`), get the number of inspections it has had.**<br><br>
**2. Get the two establishments (`DBA Name`) that have had the most inspections for each result.**<br><br>
**3. Save the results from point 2 in a new Delta table called `inspections_results`.**

---

**1. For each establishment (column `DBA Name`) and its result (column `Results`), get the number of inspections it has had.**<br><br>
This exercise can be done using either a DataFrame or a Delta Table. Considerations to keep in mind when making the choice:
- If we are working with large volumes of data and need features such as ACID transactions, version management, and Delta Log, then using a Delta Table would be highly recommended. This allows maintaining data integrity and having the capability to perform historical operations and data recovery efficiently.

- On the other hand, if our needs are more oriented towards efficiently manipulating data in memory and we do not require the persistence and advanced management offered by Delta Lake, a DataFrame would be more suitable due to its flexibility and ease of use.

In this case, I will perform the execution using both:

Option 1: Using DataFrame:

In [0]:
# Calculate the number of inspections by `DBA Name` and `Results`.
inspections_count = food_df_clean.groupBy("DBA_Name", "Results") \
                           .agg(count("*").alias("num_inspecciones")) \
                           .orderBy(col("num_inspecciones").desc())

# Show the result
inspections_count.show(truncate=False)

+----------------------------+------------------+----------------+
|DBA_Name                    |Results           |num_inspecciones|
+----------------------------+------------------+----------------+
|SUBWAY                      |Pass              |100             |
|DUNKIN DONUTS               |Pass              |42              |
|MCDONALD'S                  |Pass              |18              |
|SUBWAY                      |Out of Business   |16              |
|POTBELLY SANDWICH WORKS     |Pass              |15              |
|CHIPOTLE MEXICAN GRILL      |Pass              |15              |
|MCDONALDS                   |Pass              |11              |
|SUBWAY                      |Pass w/ Conditions|11              |
|PRET A MANGER               |Pass              |10              |
|WHOLE FOODS MARKET          |Pass              |10              |
|TACO BELL                   |Pass              |10              |
|DUNKIN DONUTS               |Out of Business   |10           

Option 2: Using the Delta Table:

In [0]:
%sql
SELECT `DBA_Name`, `Results`, COUNT(*) AS num_inspecciones
FROM food
GROUP BY `DBA_Name`, `Results`
ORDER BY `num_inspecciones` DESC;

DBA_Name,Results,num_inspecciones
SUBWAY,Pass,100
DUNKIN DONUTS,Pass,42
MCDONALD'S,Pass,18
SUBWAY,Out of Business,16
POTBELLY SANDWICH WORKS,Pass,15
CHIPOTLE MEXICAN GRILL,Pass,15
MCDONALDS,Pass,11
SUBWAY,Pass w/ Conditions,11
WHOLE FOODS MARKET,Pass,10
TACO BELL,Pass,10


**2. Get the two locations (`DBA Name`) that have had the most inspections for each of the results**<br><br>

In [0]:
from pyspark.sql.window import Window

# Define a window partitioned by 'Results' and sorted by 'num_inspections' in descending order
windowSpec = Window.partitionBy("Results").orderBy(col("num_inspecciones").desc())

# Add a ranking column based on the number of inspections per result
# Filter to get only the two most frequent locations for each result
# Sort by 'Results' and 'rank'.
ranked_inspections = inspections_count.withColumn("rank", rank().over(windowSpec))\
      .filter(col("rank") <= 2)\
      .orderBy("Results", "rank")

# Show the final result 
ranked_inspections.show(truncate=False)

+-------------------------------------------------+--------------------+----------------+----+
|DBA_Name                                         |Results             |num_inspecciones|rank|
+-------------------------------------------------+--------------------+----------------+----+
|CHINA STATION                                    |Business Not Located|1               |1   |
|SUBWAY                                           |Fail                |10              |1   |
|DUNKIN DONUTS                                    |Fail                |5               |2   |
|LANS                                             |No Entry            |5               |1   |
|LA PENA RESTAURANTE                              |No Entry            |4               |2   |
|FORK                                             |No Entry            |4               |2   |
|PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO|Not Ready           |5               |1   |
|SUBWAY                                           

In the result above, we see that some results have ties, and therefore, we get three establishments. For example, `No Entry` has `LANS` in the first position with 5 inspections, but in the second position, there is a tie between `FORK` and `LA PENA RESTAURANTE`, as each has 4 inspections with this result.

**3. Save the results from point 2 in a new Delta table called `inspections_results`.**

In [0]:
# Define the Delta Lake path for the 'inspections_results' table and delete the directory in INSPECTION_RESULTS_DELTA_PATH recursively, if it exists.
INSPECTION_RESULTS_DELTA_PATH = "/mnt/delta/inspections_results"
dbutils.fs.rm(INSPECTION_RESULTS_DELTA_PATH, recurse=True)

# Write the DataFrame to the specified Delta path
ranked_inspections.write.format("delta").save(INSPECTION_RESULTS_DELTA_PATH)

# Create a Delta table named "inspections_results" using data stored in INSPECTION_RESULTS_DELTA_PATH
spark.sql("CREATE TABLE inspections_results USING DELTA LOCATION \'" + INSPECTION_RESULTS_DELTA_PATH + "\'")

Out[25]: DataFrame[]

In [0]:
%sql
SELECT *
FROM 
  inspections_results

DBA_Name,Results,num_inspecciones,rank
CHINA STATION,Business Not Located,1,1
SUBWAY,Fail,10,1
DUNKIN DONUTS,Fail,5,2
LANS,No Entry,5,1
LA PENA RESTAURANTE,No Entry,4,2
FORK,No Entry,4,2
PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,Not Ready,5,1
SUBWAY,Not Ready,4,2
SUBWAY,Out of Business,16,1
DUNKIN DONUTS,Out of Business,10,2


# Exercise 6
1. **Update the delta table of the previous exercise `inspections_results`, specifying `DBA_Name = error`**<br>
2. **Restore the table to its original state**

---



First we will see the details of the metadata history of the `inspection_results` table:

In [0]:
%sql
DESCRIBE HISTORY inspections_results

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-07-13T13:31:28.000+0000,950217627130828,maialenigar@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(2309605403314370),0713-132713-ft8g5p6p,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14, numOutputBytes -> 1835)",null,Databricks-Runtime/12.2.x-scala2.12


Now we update the delta table by specifying `DBA_Name = Error`:

In [0]:
%sql
UPDATE inspections_results SET DBA_Name = "Error"

num_affected_rows
14


We check that it has done so:

In [0]:
%sql
SELECT *
FROM inspections_results

DBA_Name,Results,num_inspecciones,rank
Error,Business Not Located,1,1
Error,Fail,10,1
Error,Fail,5,2
Error,No Entry,5,1
Error,No Entry,4,2
Error,No Entry,4,2
Error,Not Ready,5,1
Error,Not Ready,4,2
Error,Out of Business,16,1
Error,Out of Business,10,2


We verify that this update is reflected in the history:

In [0]:
%sql
DESCRIBE HISTORY inspections_results

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2024-07-13T13:31:41.000+0000,950217627130828,maialenigar@gmail.com,UPDATE,Map(predicate -> []),null,List(2309605403314370),0713-132713-ft8g5p6p,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1835, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1308, scanTimeMs -> 134, numAddedFiles -> 1, numUpdatedRows -> 14, numAddedBytes -> 1696, rewriteTimeMs -> 1154)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-07-13T13:31:28.000+0000,950217627130828,maialenigar@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(2309605403314370),0713-132713-ft8g5p6p,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14, numOutputBytes -> 1835)",null,Databricks-Runtime/12.2.x-scala2.12


We restore the table to its original state:

In [0]:
# Read data from the Delta Lake format in the specified version '0'.
inspections_results_df_v0 = spark.read \
  .format("delta") \
  .option("versionAsOf", "0") \
  .load(INSPECTION_RESULTS_DELTA_PATH)

# Rewrite the DataFrame inspections_results_df_v0 in Delta Lake format, overwriting any existing data
inspections_results_df_v0 \
  .write \
  .format("delta") \
  .mode("overwrite") \
  .save(INSPECTION_RESULTS_DELTA_PATH)

Verify that the values have been restored correctly and check that the transaction has been recorded in the history:

In [0]:
%sql
SELECT *
FROM inspections_results

DBA_Name,Results,num_inspecciones,rank
CHINA STATION,Business Not Located,1,1
SUBWAY,Fail,10,1
DUNKIN DONUTS,Fail,5,2
LANS,No Entry,5,1
LA PENA RESTAURANTE,No Entry,4,2
FORK,No Entry,4,2
PALETERIA Y NEVERIA LA MEXICANA YOGURT AND CHURRO,Not Ready,5,1
SUBWAY,Not Ready,4,2
SUBWAY,Out of Business,16,1
DUNKIN DONUTS,Out of Business,10,2


In [0]:
%sql
DESCRIBE HISTORY inspections_results

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-07-13T13:31:50.000+0000,950217627130828,maialenigar@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2309605403314370),0713-132713-ft8g5p6p,1,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 14, numOutputBytes -> 1835)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-07-13T13:31:41.000+0000,950217627130828,maialenigar@gmail.com,UPDATE,Map(predicate -> []),null,List(2309605403314370),0713-132713-ft8g5p6p,0,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1835, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 1308, scanTimeMs -> 134, numAddedFiles -> 1, numUpdatedRows -> 14, numAddedBytes -> 1696, rewriteTimeMs -> 1154)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-07-13T13:31:28.000+0000,950217627130828,maialenigar@gmail.com,WRITE,"Map(mode -> ErrorIfExists, partitionBy -> [])",null,List(2309605403314370),0713-132713-ft8g5p6p,null,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 14, numOutputBytes -> 1835)",null,Databricks-Runtime/12.2.x-scala2.12


# Exercise 7

**Create a Structured Streaming application that reads data from the Kafka topic `inspections`. The Kafka server URL is `35.237.99.179:9094`:**

**The data from this topic is exactly the same as what we have been analyzing throughout this notebook, `Food Inspections`, so the schema is the same.**

In [0]:
# Read streaming data from Kafka
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "35.237.99.179:9094") \
  .option("subscribe", "inspections") \
  .load()

# Print the schema of the DataFrame
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [0]:

# Define the schema to parse JSON data
schema = StructType(
  [
    StructField("Inspection ID", StringType(), True),
    StructField("DBA Name", StringType(), True),
    StructField("AKA Name", StringType(), True),
    StructField("License #", StringType(), True),
    StructField("Facility Type", StringType(), True),
    StructField("Risk", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Zip", StringType(), True),
    StructField("Inspection Date", StringType(), True),
    StructField("Inspection Type", StringType(), True),
    StructField("Results", StringType(), True),
    StructField("Violations", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True),
    StructField("Location", StringType(), True)
  ]
)

# Select the columns key, value, and timestamp, converting key and value to strings
# Parse the value column from JSON format using the specified schema
# Select the key, timestamp, and all columns from the parsed JSON value.
dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

# Print the schema
dataset.printSchema()

root
 |-- key: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- Inspection ID: string (nullable = true)
 |-- DBA Name: string (nullable = true)
 |-- AKA Name: string (nullable = true)
 |-- License #: string (nullable = true)
 |-- Facility Type: string (nullable = true)
 |-- Risk: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Inspection Date: string (nullable = true)
 |-- Inspection Type: string (nullable = true)
 |-- Results: string (nullable = true)
 |-- Violations: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



Start the streaming with `writeStream`. `writeStream` configures a streaming channel in Spark Structured Streaming and initiates the execution of the streaming. The configuration applied in this case includes: append mode, storing results in memory, assigning a query name, etc.

This enables real-time data processing and analysis, making the results immediately available for querying through Spark SQL or other subsequent applications.

In [0]:
# Specify the output mode as 'append' (only new rows added to the results table)
# Define the output sink format as 'memory' (store the results table in memory)
# Option to truncate long strings in the output table (set to 'false' to display the full content)
# Assign a name to the query (to be referenced in Spark SQL)
# Start the streaming query
dataset.writeStream \
 .outputMode("append") \
 .format("memory") \
 .option("truncate", "false") \
 .queryName("inspections_topic") \
 .start()

Out[37]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fde04c4bfd0>

We check the availability of the data by running a query in Spark SQL:

In [0]:
%sql
SELECT
  *
FROM
  inspections_topic

key timestamp Inspection ID DBA Name AKA Name License # Facility Type Risk Address City State Zip Inspection Date Inspection Type Results Violations Latitude Longitude Location null 2024-07-13T13:35:46.231+0000 2285434 ORA CASUAL DINING ORA CASUAL DINING 2652875 Restaurant Risk 1 (High) 6043 W ADDISON ST CHICAGO IL 60634 04/19/2019 License Re-Inspection Pass 41.945644781746616 -87.778210263651 (-87.778210263651, 41.945644781746616) null 2024-07-13T13:35:46.233+0000 2285430 LOST LARSON LOST LARSON 2632242 Restaurant Risk 2 (Medium) 5318 N CLARK ST CHICAGO IL 60640 04/18/2019 License Re-Inspection Pass 41.97868454005666 -87.6684667325925 (-87.6684667325925, 41.97868454005666) null 2024-07-13T13:35:46.235+0000 2285424 LOST LARSON LOST LARSON 2583423 Restaurant Risk 1 (High) 5318 N CLARK ST CHICAGO IL 60640 04/18/2019 Canvass Re-Inspection Pass 23. PROPER DATE MARKING AND DISPOSITION - Comments: OBSERVED NO DATE MARKING SYSTEM IN PLACE. READY TO EAT AND OR TIME TEMPERATURE CONTROLLED FOODS PREPARED AND HELD IN A RETAIL FOOD ESTABLISHMENT FOR OVER 24 HOURS MUST HAVE A CLEARLY MARKED LABELING SYSTEM IN PLACE TO INDICATE THE DATE OR DAY BY WHICH THE FOOD SHALL BE CONSUMED ON PREMISES, SOLD, OR DISCARDED WHEN HELD AT A TEMPERATURE OF 41F OR LESS FOR A MAXIMUM OF 7 DAYS WITH THE DATE OF PREPARATION COUNTING AS DAY 1. PRIORITY FOUNDATION VIOLATION 7-38-005 NO CITATION ISSUED. 41.97868454005666 -87.6684667325925 (-87.6684667325925, 41.97868454005666) null 2024-07-13T13:35:46.236+0000 2285421 NICO NICO 2271049 Restaurant Risk 1 (High) 1015 N RUSH ST CHICAGO IL 60611 04/18/2019 Canvass Pass w/ Conditions 2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - Comments: - THE DESIGNATED PERSON IN CHARGE, WHO IS ON SITE, MUST OBTAIN AND MAINTAIN A VALID CITY OF CHICAGO FOOD SERVICE SANITATION LICENSE. PRIORITY FOUNDATION VIOLATION#:7-38-012. NO CITATION ISSUED. | 3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: - NO VERIFIABLE EMPLOYEE HEALTH POLICY ON SITE. MUST PROVIDE. PRIORITY FOUNDATION VIOLATION#: 7-38-010. NO CITATION ISSUED. | 10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLIED AND ACCESSIBLE - Comments: - OBSERVED NO HANDWASH SIGNAGE AT THE HANDWASH SINK IN THE EMPLOYEE SERVING/DINING AREA. MUST PROVIDE HANDWASH SIGNAGE AT ALL HANDWASH SINKS AND MAINTAIN. | 14. REQUIRED RECORDS AVAILABLE: SHELLSTOCK TAGS, PARASITE DESTRUCTION - Comments: - OBSERVED 1# OF RAW OYSTERS IN A COOLER DRAWER WITHOUT ANY SHELLFISH TAGS OR IDENTIFICATION ATTACHED. MUST PROVIDE SHELLFISH TAGS/LABELS TO BE HELD WITH ALL SHELLSTOCK DURING STORAGE. FACILITY CORRECTED VIOLATION ON SITE BY DISCARDING THE OYSTERS WITH A REPORTED TOTAL VALUE OF $30. PRIORITY FOUNDATION VIOLATION#:7-38-005. CITATION ISSUED. | 16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED - Comments: - OBSERVED A SLIGHT DEBRIS BUILDUP ON THE INTERIOR OF ALL ICE MACHINES. MUST DETAIL CLEAN/HAVE SERVICED AND MAINTAIN. | 22. PROPER COLD HOLDING TEMPERATURES - Comments: OBSERVED THE FOLLOWING FOODS HELD AT IMPROPER TEMPERATURES: 1/2# OF SOFT BOILED EGGS IN THE 1ST FLOOR PREP COOLER (48.4F), 5# SOFT CHEESE IN THE PREP KITCHEN COOLER DRAWER (46.4F). FACILITY CORRECTED VIOLATION ON SITE BY DISCARDING THE FOODS WITH A REPORTED TOTAL VALUE OF $75. PRIORITY VIOLATION#:7-38-005. CITATION ISSUED. | 23. PROPER DATE MARKING AND DISPOSITION - Comments: - MUST PROVIDE PROPER LABELS FOR DATE MARKING THE PREPARATION AND CONSUME BY/DISCARD DATE OF ALL REFRIGERATED, READY-TO-EAT, TCS FOODS HELD OVER 24HRS. PRIORITY FOUNDATION VIOLATION#: 7-38-005. NO CITATION ISSUED. | 25. CONSUMER ADVISORY PROVIDED FOR RAW/UNDERCOOKED FOOD - Comments: - MUST PROVIDE CONSUMER ADVISORY (ON SITE), REMINDER (IN THE MENU), AND DISCLOSURE (BY THE FOOD ITEMS) FOR ALL UNDERCOOKED FOODS. PRIORITY FOUNDATION VIOLATION#: 7-38-005. NO CITATION ISSUED. | 38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - Comments: - OBSERVED A 1/2' GAP AT THE BOTTOM OF THE BELLEVUE FRONT ENTRANCE DOORS. MUST PROV

# Exercise 8
**Based on the data source from the previous exercise, obtain the number of inspections by `Facility Type` every 5 seconds.**

We will group the data by time windows and then count the records by `Facility Type`. We will use the `.display()` method for visualization.

**Note:** The `.display()` method is only available in certain development environments, such as Databricks, and is primarily designed for rapid data exploration and development. It is not recommended for use in production environments.



In [0]:
dataset.groupBy(window(col("timestamp"), "5 seconds"), col("Facility Type")) \
    .count() \
    .display()

window,Facility Type,count
"List(2024-07-13T13:37:05.000+0000, 2024-07-13T13:37:10.000+0000)",Catering,1
"List(2024-07-13T13:37:45.000+0000, 2024-07-13T13:37:50.000+0000)",Restaurant,6
"List(2024-07-13T13:37:25.000+0000, 2024-07-13T13:37:30.000+0000)",Restaurant,7
"List(2024-07-13T13:38:45.000+0000, 2024-07-13T13:38:50.000+0000)",Restaurant,7
"List(2024-07-13T13:37:05.000+0000, 2024-07-13T13:37:10.000+0000)",Restaurant,6
"List(2024-07-13T13:39:25.000+0000, 2024-07-13T13:39:30.000+0000)",Catering,1
"List(2024-07-13T13:36:25.000+0000, 2024-07-13T13:36:30.000+0000)",Restaurant,7
"List(2024-07-13T13:39:05.000+0000, 2024-07-13T13:39:10.000+0000)",Children's Services Facility,1
"List(2024-07-13T13:37:25.000+0000, 2024-07-13T13:37:30.000+0000)",Special Event,1
"List(2024-07-13T13:39:05.000+0000, 2024-07-13T13:39:10.000+0000)",Restaurant,6


Instead of using `.display()` in production environments, it is preferable to save the results in persistent storage formats using the `writeStream` method.

Below is the code demonstrating how to do this with `writeStream` for production environments:

In [0]:
# Specify the output mode as 'update'
# Define the output sink format as 'memory'
# Set the option to truncate long strings in the output table to false
# Assign a name to the query (to be referenced in Spark SQL)
# Start the streaming query
dataset.groupBy(window(col("timestamp"), "5 seconds"), col("Facility Type")) \
    .count() \
    .writeStream \
    .outputMode("update") \
    .format("memory") \
    .option("truncate", "false") \
    .queryName("inspections_grouped_topic") \
    .start()

Out[40]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fde04c49f40>

As in exercise 7, we check the availability of the data by running a query in Spark SQL:

In [0]:
%sql
SELECT
  *
FROM
  inspections_grouped_topic

window,Facility Type,count
"List(2024-07-13T13:37:05.000+0000, 2024-07-13T13:37:10.000+0000)",Catering,1
"List(2024-07-13T13:37:05.000+0000, 2024-07-13T13:37:10.000+0000)",Restaurant,6
"List(2024-07-13T13:36:45.000+0000, 2024-07-13T13:36:50.000+0000)",Liquor,1
"List(2024-07-13T13:36:45.000+0000, 2024-07-13T13:36:50.000+0000)",Mobile Food Preparer,1
"List(2024-07-13T13:36:45.000+0000, 2024-07-13T13:36:50.000+0000)",Restaurant,8
"List(2024-07-13T13:37:05.000+0000, 2024-07-13T13:37:10.000+0000)",Grocery Store,3
"List(2024-07-13T13:37:25.000+0000, 2024-07-13T13:37:30.000+0000)",Restaurant,7
"List(2024-07-13T13:37:25.000+0000, 2024-07-13T13:37:30.000+0000)",Special Event,1
"List(2024-07-13T13:37:25.000+0000, 2024-07-13T13:37:30.000+0000)",School,2
"List(2024-07-13T13:37:45.000+0000, 2024-07-13T13:37:50.000+0000)",Restaurant,6


# Exercise 9
**Based on the data source from exercise 7, obtain the number of inspections by `Results` every 5 seconds for the last 30 seconds.**

In [0]:
dataset.groupBy(window(col("timestamp"), "30 seconds", "5 seconds"), col("Results")) \
    .count() \
    .writeStream \
    .outputMode("update") \
    .format("memory") \
    .option("truncate", "false") \
    .queryName("inspections_grouped_topic2") \
    .start()

Out[42]: <pyspark.sql.streaming.query.StreamingQuery at 0x7fde04ba55b0>

As in exercise 8, we check the availability of the data by running a query in Spark SQL:

In [0]:
%sql
SELECT
  *
FROM
  inspections_grouped_topic2

window,Results,count
"List(2024-07-13T13:39:40.000+0000, 2024-07-13T13:40:10.000+0000)",Fail,3
"List(2024-07-13T13:39:25.000+0000, 2024-07-13T13:39:55.000+0000)",No Entry,1
"List(2024-07-13T13:39:20.000+0000, 2024-07-13T13:39:50.000+0000)",Pass,10
"List(2024-07-13T13:39:15.000+0000, 2024-07-13T13:39:45.000+0000)",No Entry,1
"List(2024-07-13T13:39:30.000+0000, 2024-07-13T13:40:00.000+0000)",Out of Business,1
"List(2024-07-13T13:39:00.000+0000, 2024-07-13T13:39:30.000+0000)",Pass w/ Conditions,1
"List(2024-07-13T13:39:25.000+0000, 2024-07-13T13:39:55.000+0000)",Out of Business,3
"List(2024-07-13T13:39:10.000+0000, 2024-07-13T13:39:40.000+0000)",Pass,6
"List(2024-07-13T13:39:20.000+0000, 2024-07-13T13:39:50.000+0000)",Out of Business,3
"List(2024-07-13T13:39:20.000+0000, 2024-07-13T13:39:50.000+0000)",Fail,3


# Exercise 10
1. **Update the `Results` column of the Delta table for food inspections created in exercise 1 to the value `No result`.**
2. **Update the data in the modified table from point 1 as new items arrive in Kafka.**
---

It is advisable to stop all previous streams, as the one for this exercise tends to be resource-intensive.

**1. Update the `Results` column of the Delta table for food inspections created in exercise 1 to the value `No result`.**

Before starting, we print the details of the Delta `food` table:

In [0]:
%sql
DESCRIBE FORMATTED food

col_name,data_type,comment
Inspection_ID,int,null
DBA_Name,string,null
AKA_Name,string,null
License_number,int,null
Facility_Type,string,null
Risk,string,null
Address,string,null
City,string,null
State,string,null
Zip,int,null


And we also print an example of the values ​​it contains:

In [0]:
%sql
SELECT *
FROM 
  food
LIMIT 3

Inspection_ID,DBA_Name,AKA_Name,License_number,Facility_Type,Risk,Address,City,State,Zip,Inspection_Date,Inspection_Type,Results,Violations,Latitude,Longitude,Location
2373596,COUNTY BBQ,COUNTY BBQ,2732781,Restaurant,Risk 3 (Low),1352 W TAYLOR ST,CHICAGO,IL,60607,2020-06-11,License,Fail,null,41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"
2373587,KIDZ CREATIVE CORNER,KIDZ CREATIVE CORNER,2555611,Daycare Above and Under 2 Years,Risk 1 (High),4259 N WESTERN AVE,CHICAGO,IL,60618,2020-06-11,License Re-Inspection,Pass,null,41.95944802136064,-87.68848155320953,"(-87.68848155320953, 41.95944802136064)"
2373578,DSD DELI,DSD DELI,2626186,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE,CHICAGO,IL,60630,2020-06-11,License Re-Inspection,Pass,null,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"


We will now update the `Results` column to set the value `No result`:

In [0]:
%sql
-- Update the column Results
UPDATE food SET Results = 'No result';

num_affected_rows
9995


We check that it has been updated by printing a sample of the data again:

In [0]:
%sql
SELECT *
FROM 
  food
LIMIT 10

Inspection_ID,DBA_Name,AKA_Name,License_number,Facility_Type,Risk,Address,City,State,Zip,Inspection_Date,Inspection_Type,Results,Violations,Latitude,Longitude,Location
2373596,COUNTY BBQ,COUNTY BBQ,2732781,Restaurant,Risk 3 (Low),1352 W TAYLOR ST,CHICAGO,IL,60607,2020-06-11,License,No result,null,41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"
2373587,KIDZ CREATIVE CORNER,KIDZ CREATIVE CORNER,2555611,Daycare Above and Under 2 Years,Risk 1 (High),4259 N WESTERN AVE,CHICAGO,IL,60618,2020-06-11,License Re-Inspection,No result,null,41.95944802136064,-87.68848155320953,"(-87.68848155320953, 41.95944802136064)"
2373578,DSD DELI,DSD DELI,2626186,DELI/GROCERY,Risk 3 (Low),5205 N MILWAUKEE,CHICAGO,IL,60630,2020-06-11,License Re-Inspection,No result,null,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"
2373575,DSD DELI,DSD DELI,2626178,DELI/GROCERY,Risk 1 (High),5205 N MILWAUKEE,CHICAGO,IL,60630,2020-06-11,License Re-Inspection,No result,null,41.97484853157327,-87.76682905043768,"(-87.76682905043768, 41.97484853157327)"
2373576,COUNTY BBQ,COUNTY BBQ,2732780,Restaurant,Risk 1 (High),1352 W TAYLOR ST,CHICAGO,IL,60607,2020-06-11,License,No result,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: NO EMPLOYEE HEALTH POLICY ON PREMISES AS REQUIRED; MUST PROVIDE. INSTRUCTED THE PERSON IN CHARGE TO PROVIDE DOCUMENTATION OF EMPLOYEES RESPONSIBILITY TO REPORT ANY ILLNESS TRANSMISSIBLE THROUGH FOOD. PRIORITY FOUNDATION VIOLATION 7-38-010. | 16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED - Comments: THE LOW TEMP DISH MACHINE IN THE PREP AREA IS NOT SANITIZING PROPERLY; MUST CORRECT. FOUND NO CHEMICAL READING WITH THE TEST STRIP AFTER TWO CYCLES. MACHINE MUST BE ABLE TO PROPERLY SANITIZE AT 100PPM CHLORINE. PRIORITY VIOLATION 7-38-025. | 51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICES - Comments: OBSERVED THE EXPOSED HANDSINK IN THE BASEMENT PREP AREA DRAINING SLOWLY; MUST CORRECT. | 57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAINING - Comments: INSTRUCTED THE PERSON IN CHARGE THAT ALL FOOD HANDLERS MUST OBTAIN THE REQUIRED FOOD HANDLER TRAINING.",41.86945393993995,-87.66133829204541,"(-87.66133829204541, 41.86945393993995)"
2373547,NEW SEOUL,NEW SEOUL,62062,null,Risk 3 (Low),5351 N LINCOLN AVE,CHICAGO,IL,60625,2020-06-11,Canvass,No result,null,41.97923607630261,-87.69252686676323,"(-87.69252686676323, 41.97923607630261)"
2373543,"CAFE UTJEHA, INC.",CAFE UTJEHA,1592606,Restaurant,Risk 2 (Medium),5350 N LINCOLN AVE,CHICAGO,IL,60625,2020-06-11,Canvass,No result,"2. CITY OF CHICAGO FOOD SERVICE SANITATION CERTIFICATE - Comments: NO CITY OF CHICAGO CERTIFIED FOOD MANAGER ON SITE DURING THE INSPECTION WHILE THE TEMPERATURE CONTROL FOR SAFETY FOODS (MILK) HAS BEEN PREPARED AND SERVED. INSTRUCTED A CITY OF CHICAGO CERTIFIED FOOD MANAGER MUST BE ON SITE AT ALL TIMES FOODS ARE BEING PREPARED AND SERVED. PRIORITY FOUNDATION VIOLATION 7-38-012.CITATION ISSUED | 3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL EMPLOYEE; KNOWLEDGE, RESPONSIBILITIES AND REPORTING - Comments: MANAGER PROVIDED NO PROOF OF EMPLOYEE HEALTH POLICY ON SITE. INSTRUCTED TO PROVIDE AN EMPLOYEE HEALTH POLICY THAT INCLUDES A SIGNED ACKNOWLEDGEMENT FROM EACH EMPLOYEE. PRIORITY FOUNDATION. 7.38.010. NO CITATION ISSUED. | 36. THERMOMETERS PROVIDED & ACCURATE - Comments: OBSERVED NO PROBE THERMOMETER FOR TAKING FOOD TEMPERATURES.MUST PROVIDE.(PRIORITY FOUNDATION-NO CITATION ISSUED) 7-38-005 | 44. UTENSILS, EQUIPMENT & LINENS: PROPERLY STORED, DRIED, & HANDLED - Comments: MUST REMOVE SINGLE SERVICE FOOD UTENSILS AND CUPS STORED IN KITCHEN CABINETS UNDER HAND WASHING SINK | 51. PLUMBING INSTALLED; PROPER BACKFLOW DEVICES - Comments: BACKFLOW PREVENTION DEVICES NOT LOCATED CAPPUCCINO/ESPRESSO MACHINE. MUST INSTALL BACKFLOW PREVENTION DEVICES OR AIR GAP BETWEEN THE WATER SUPPLY INLET AND THE UNIT SO IT MAY BE LOCATED TO BE SERVICED AND MAINTAINED | 55. PHYSICAL FACILITIES INSTALLED, M

**2. Update the data in the modified table from point 1 as new items arrive in Kafka**

First we establish the connection to Kafka by reusing the code from exercise 7:

In [0]:
# Read streaming data from Kafka
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "35.237.99.179:9094") \
  .option("subscribe", "inspections") \
  .load()


# Define the schema for parsing JSON data
schema = StructType(
  [
    StructField("Inspection ID", StringType(), True),
    StructField("DBA Name", StringType(), True),
    StructField("AKA Name", StringType(), True),
    StructField("License #", StringType(), True),
    StructField("Facility Type", StringType(), True),
    StructField("Risk", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("City", StringType(), True),
    StructField("State", StringType(), True),
    StructField("Zip", StringType(), True),
    StructField("Inspection Date", StringType(), True),
    StructField("Inspection Type", StringType(), True),
    StructField("Results", StringType(), True),
    StructField("Violations", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True),
    StructField("Location", StringType(), True)
  ]
)

# Convert Kafka's data to the schema. To do so:
#   - Select key, value, and timestamp columns, converting key and value to strings
#   - Parse the value column from the JSON format using the specified schema
#   - Select the key, the timestamp and all columns of the parsed JSON value
dataset = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)", "timestamp") \
    .withColumn("value", from_json("value", schema)) \
    .select(col('key'), col("timestamp"), col('value.*'))

In this exercise, it is essential to consider that the Delta table schema to be modified contains columns typed as integers, doubles, among others. Therefore, it is necessary to properly typify the data read from Kafka. For this purpose, the following actions will be performed:

In [0]:
from pyspark.sql.types import IntegerType, DoubleType, DateType

# Make the casting of the data types to avoid problems in the merge
dataset \
  .withColumn("Inspection ID", col("Inspection ID").cast(IntegerType())) \
  .withColumn("License #", col("License #").cast(IntegerType())) \
  .withColumn("Zip", col("Zip").cast(IntegerType())) \
  .withColumn("Inspection Date", to_date(col("Inspection Date"), "MM/dd/yyyy")) \
  .withColumn("Latitude", col("Latitude").cast(DoubleType())) \
  .withColumn("Longitude", col("Longitude").cast(DoubleType()))

Out[50]: DataFrame[key: string, timestamp: timestamp, Inspection ID: int, DBA Name: string, AKA Name: string, License #: int, Facility Type: string, Risk: string, Address: string, City: string, State: string, Zip: int, Inspection Date: date, Inspection Type: string, Results: string, Violations: string, Latitude: double, Longitude: double, Location: string]

Now we will define a function to update the Delta table:

In [0]:
from delta.tables import *

# Adjust the number of partitions for shuffle operations, optimizing performance based on cluster size
spark.conf.set("spark.sql.shuffle.partitions", "200")  


# Function for updating the Delta table
def upsertToDelta(microBatchOutputDF, batchId):

    # Debugging: Print the number of records in the batch
    print(f"Processing batch ID: {batchId} with {microBatchOutputDF.count()} records.")
    
    # Load existing Delta table
    delta_table = DeltaTable.forName(spark, "food")

    # Debugging: Check that the Delta Table has been loaded successfully
    record_count = delta_table.toDF().count()
    print(f"Total number of records in the table 'food': {record_count}")

    # Debugging: Filter the microbatch records that already exist in the 'food' table and count how many there are
    matched_count = microBatchOutputDF.filter("`Inspection ID` IN (SELECT Inspection_ID FROM food)").count()
    print(f"Matching records: {matched_count}")

    # Perform merge operation on Delta table
    delta_table.alias("target").merge(
        microBatchOutputDF.alias("source"),
        "target.Inspection_ID = cast(source.`Inspection ID` as Integer)"
    ).whenMatchedUpdate(set = {
            "target.Results": "source.Results"
    }).whenNotMatchedInsert(values={
            "target.Inspection_ID": "source.`Inspection ID`",
            "target.DBA_Name": "source.`DBA Name`",
            "target.AKA_Name": "source.`AKA Name`",
            "target.License_number": "source.`License #`",
            "target.Facility_Type": "source.`Facility Type`",
            "target.Risk": "source.Risk",
            "target.Address": "source.Address",
            "target.City": "source.City",
            "target.State": "source.State",
            "target.Zip": "source.Zip",
            "target.Inspection_Date": "source.`Inspection date`",
            "target.Results": "source.Results",
            "target.Violations": "source.Violations",
            "target.Latitude": "source.Latitude",
            "target.Longitude": "source.Longitude",
            "target.Location": "source.Location"         
    }).execute()


    # Debugging: Convert the Delta table to a DataFrame and counts the total number of records in the resulting DataFrame.
    delta_table_a_df = delta_table.toDF()
    record_count = delta_table_a_df.count()
    print(f"Merge completed. Total number of records in the table 'food' after the merge: {record_count}")

    # Debugging: Count the values in the 'Results' column and display the result
    result_count = delta_table_a_df.groupBy("Results").count()
    result_count.show(truncate=False)

Apply the `upsertToDelta` function to the stream:

In [0]:
# Delete the checkpoints directory for the 'food' table, including all its contents.
dbutils.fs.rm("dbfs:/mnt/delta/checkpoints/food", recurse=True)

# Configure the stream to process data and apply upsertToDelta function
query = dataset.writeStream \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .option("checkpointLocation", "dbfs:/mnt/delta/checkpoints/food") \
    .start()

We will check that the updates are being made by counting the values of the "Results" column. Executing it twice, we will see that in the first execution the count of the results different to "No result" is higher, and in the second one lower.

In [0]:
%sql
SELECT Results, COUNT(*) as Count
FROM food
GROUP BY Results

Registros coincidentes: 0


Results,Count
No result,9995


In [0]:
%sql
SELECT Results, COUNT(*) as Count
FROM food
GROUP BY Results

Results,Count
No result,9974
No Entry,3
Pass w/ Conditions,4
Out of Business,4
Pass,9
Fail,1


We also check the total number of records, to see if new ones have been added:

In [0]:
%sql
SELECT COUNT(*) AS total_lines FROM food

total_lines
9995


**Clarification on all debugging instructions** incorporated in the `upsertToDelta` method:

During the development of the merge method, I encountered several challenges:
- It has been difficult to correctly map column names.
- Once the mapping was completed, the execution did not update the data (due to incorrect filtering), and I couldn't identify the apparent reason for this issue.

To detect the cause of the failures, it was necessary to include various debugging instructions, such as:

```python
# Debugging: Print the number of records in the batch
print(f"Processing batch ID: {batchId} with {microBatchOutputDF.count()} records.")

# Debugging: Check that the Delta Table has been loaded successfully
record_count = delta_table.toDF().count()
print(f"Total number of records in the table 'food': {record_count}")

# Debugging: Filter the microbatch records that already exist in the 'food' table and count how many there are
matched_count = microBatchOutputDF.filter("`Inspection ID` IN (SELECT Inspection_ID FROM food)").count()
print(f"Matching records: {matched_count}")

...
```

The results of these instructions can be viewed in the Standard Output (stdout) of Databricks by accessing the menu:

- `Compute` > Select the cluster number > `Driver Logs` tab > `Standard Output` section.

This has allowed me to monitor the execution and effectively debug the process.
